In [2]:
#IMPORTING DEPENDENCIES

from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [4]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [13]:
# Scrape NASA Mars News
browser = init_browser()

# Visit NASA Mars News site
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

# Find news slide
news_slide = soup.find('li', class_='slide')

# Scrape slide title
news_title = news_slide.find('div', class_='content_title').get_text()

# Scrape slide text
news_preview  = news_slide.find('div', class_='article_teaser_body').get_text()

# Store data in a dictionary
news_data = {
    
    "title": news_title,
    "content": news_preview
}

# Close the browser after scraping
browser.quit()

In [ ]:
# Scrape JPL Mars Featured Images
browser = init_browser()

# Visit NASA Mars News site
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(3)

# Scrape page into Soup
html = browser.html.links.find_by_partial_text('FULL IMAGE').click()
soup = bs(html, "html.parser")

#browser.links.find_by_partial_text('FULL IMAGE').click()

# Find news slide
img_url = soup('img')
#jpl_url = "https://www.jpl.nasa.gov" + img_url

# Close the browser after scraping
browser.quit()

In [47]:
img_url

[<img alt="" class="print_only print_logo" src="/assets/images/logo_nasa_trio_black@2x.png"/>,
 <img alt="" class="print_only print_logo" src="/assets/images/logo_nasa_trio_black@2x.png"/>,
 <img alt="more arrow" src="/assets/images/overlay-arrow.png"/>,
 <img alt="Her Desher Vallis" class="thumb" src="/spaceimages/images/wallpaper/PIA24088-640x350.jpg" title="Her Desher Vallis"/>,
 <img alt="more arrow" src="/assets/images/overlay-arrow.png"/>,
 <img alt="Sirenum Fossae" class="thumb" src="/spaceimages/images/wallpaper/PIA24087-640x350.jpg" title="Sirenum Fossae"/>,
 <img alt="more arrow" src="/assets/images/overlay-arrow.png"/>,
 <img alt="Daedalia Planum" class="thumb" src="/spaceimages/images/wallpaper/PIA24086-640x350.jpg" title="Daedalia Planum"/>,
 <img alt="more arrow" src="/assets/images/overlay-arrow.png"/>,
 <img alt="Gale Crater" class="thumb" src="/spaceimages/images/wallpaper/PIA24085-640x350.jpg" title="Gale Crater"/>,
 <img alt="more arrow" src="/assets/images/overlay-a